1. Load the Model & Feature Extraction Parameters

In [3]:
from joblib import load
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, hog

# Load the trained SVM model and feature params
model = load('best_svm_gender_model.pkl')
params = load('feature_extraction_params.pkl')

D:\VB\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
D:\VB\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
D:\VB\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [5]:
def process_new_image(image):
    # Convert to grayscale if needed
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize to the expected input size
    image = cv2.resize(image, params['preprocessing']['target_size'])
    
    # Preprocessing (CLAHE + Gaussian Blur)
    clahe = cv2.createCLAHE(**params['preprocessing']['clahe_params'])
    image = clahe.apply(image)
    image = cv2.GaussianBlur(image, params['preprocessing']['gaussian_blur'], 0)
    
    # Compute LBP features
    lbp_image = local_binary_pattern(image, **params['lbp_params'])
    n_bins = params['lbp_params']['P'] + 2
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=n_bins, range=(0, n_bins))
    lbp_hist = lbp_hist.astype('float32')
    lbp_hist = (lbp_hist + 1e-7) / (np.sum(lbp_hist) + 1e-7)
    
    # Compute HOG features
    hog_features = hog(image, **params['hog_params'])
    
    # Combine features
    combined_features = np.concatenate([lbp_hist, hog_features])
    return combined_features

In [ ]:
import os

def test_on_dataset(dataset_dir, true_label):
    correct = 0
    total = 0
    
    for filename in os.listdir(dataset_dir):
        if filename.lower().endswith(('jpg', 'jpeg', 'png')):
            img_path = os.path.join(dataset_dir, filename)
            image = cv2.imread(img_path)
            
            try:
                features = process_new_image(image)
                prediction = model.predict([features])[0]
                
                if prediction == true_label:
                    correct += 1
                total += 1
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    
    accuracy = (correct / total) * 100
    print(f"Accuracy on {dataset_dir}: {accuracy:.2f}%")

# Example usage
test_on_dataset('path_to_male_test_images', 0)   # Male label = 0
test_on_dataset('path_to_female_test_images', 1) # Female label = 1